In [5]:
import sys, os, glob
import pandas as pd
import statsmodels.api as sm
import numpy as np
import tqdm.notebook as tqdm

from PIL import Image
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd


sys.path.append('../')

from utils.GetFileNames import get_file_names
from utils.Metrics import iou

In [6]:
# get accuracy per image for each model
base_path = '../data/test/{}_test_predictions_by_pod/'

vanilla_path = base_path.format('vanilla')
up_path = base_path.format('up')
down_path = base_path.format('down')

gt_path = '../data/test/test_masks_by_pod/'

# only need one set of names since they are the same for all models
gt_names = get_file_names(gt_path)

In [ ]:
# load the data
vanilla_preds, up_preds, down_preds, gt_masks = [], [], [], []

print('Loading data...')
for name in tqdm(gt_names):
    vanilla_preds.append(np.array(Image.open(vanilla_path + name)) / 255)
    up_preds.append(np.array(Image.open(up_path + name)) / 255)
    down_preds.append(np.array(Image.open(down_path + name)) / 255)
    gt_masks.append(np.array(Image.open(gt_path + name)) / 255)
print("Done.")

In [ ]:
# get accuracies for each model
vanilla_seed_acc, up_seed_acc, down_seed_acc = [], [], []

print("Calculating accuracies...")
for i in tqdm(range(len(gt_masks))):
    vanilla, up, down, mask = vanilla_preds[i], up_preds[i], down_preds[i], gt_masks[i]

    vanilla_seed = vanilla[:, :, 2]
    up_seed = up[:, :, 2]
    down_seed = down[:, :, 2]
    mask_seed = mask[:, :, 2]

    vanilla_seed_acc.append(iou(vanilla_seed, mask_seed))
    up_seed_acc.append(iou(up_seed, mask_seed))
    down_seed_acc.append(iou(down_seed, mask_seed))
print("Done.")
    

# Seed Model Significance Test

In [ ]:
# create dataframe
seed_df = pd.DataFrame({
    'vanilla_seed': vanilla_seed_acc,
    'upweighted_seed': up_seed_acc,
    'downweighted_seed': down_seed_acc
})
seed_df_melt = pd.melt(seed_df.reset_index(), id_vars=['index'], value_vars=['vanilla', 'upweighted', 'downweighted'])
seed_df_melt.columns = ['index', 'treatments', 'value']

In [ ]:
# run ANOVA
model = ols('value ~ C(treatments)', data=seed_df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)
print()

In [ ]:
# run tukey HSD test
m_comp = pairwise_tukeyhsd(endog=seed_df_melt['value'], groups=seed_df_melt['treatments'], alpha=0.05)
print(m_comp)